In [ ]:
# 1) (Optional) Installs — uncomment on first run
# !pip install -U matplotlib timm torchvision torch tqdm scikit-learn --quiet
!pip install -U open-clip-torch torchinfo --quiet
print('If you need packages, uncomment the pip lines above and run this cell.')

In [ ]:
# 2) Common imports + seeds
import math, random, os, time
from dataclasses import dataclass
from typing import Tuple, List

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset

import torchvision
from torchvision import datasets, transforms

import torchinfo

import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import open_clip

def set_seed(seed:int=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False

set_seed(123)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Device is {device}")

In [ ]:
from dataclasses import dataclass
from typing import Tuple

# Pick a lightweight variant for quick experiments
MODEL_NAME = 'MobileCLIP2-S0'   # alternatives: 'MobileCLIP2-S2', 'MobileCLIP2-B', 'MobileCLIP2-S3', 'MobileCLIP2-S4', 'MobileCLIP2-L-14'
PRETRAINED = 'dfndr2b'          # per MobileCLIP2 release in OpenCLIP


@dataclass
class Config:
    model_name: str = "ViT-B-32"  # new MobileCLIP2 in open_clip
    pretrained: str = "laion2b_s34b_b79k"
    image_size: int = 224
    batch_size: int = 256
    num_workers: int = 4
    max_epochs: int = 1

    # LR split: base vs prompt
    lr_base: float = 1e-5
    lr_prompt: float = 5e-3
    weight_decay: float = 0.05

    # Unfreeze (if you want a light finetune on top of prompt)
    unfreeze_layers: Tuple[str, ...] = tuple()  # e.g. ("visual.transformer.resblocks.11",)

    # CoOp prompt length
    prompt_len: int = 16

    # Overfit sanity tiny subset
    overfit_n_classes: int = 2
    overfit_k_per_class: int = 8
    overfit_epochs: int = 50


cfg = Config()
cfg


In [ ]:
# --- Load model + preprocess ---
model, _, preprocess = open_clip.create_model_and_transforms(cfg.model_name, pretrained=cfg.pretrained)
tokenizer = open_clip.get_tokenizer(cfg.model_name)
model = model.to(device).eval()

torchinfo.summary(model)

In [ ]:
full_train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=preprocess)
full_train_size = len(full_train_ds)
val_size = int(full_train_size * 0.1)

train_ds, val_ds = torch.utils.data.random_split(full_train_ds, [full_train_size - val_size, val_size])

train_loader = DataLoader(train_ds, batch_size=128, shuffle=False, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2)

classnames = full_train_ds.classes
num_classes = len(classnames)

print(f"Val dataset:   {len(val_ds)} images, {num_classes} classes")

def show_batch(dl: DataLoader, rows: int=2, cols: int=8):
    batch = next(iter(dl))
    images, labels = batch
    fig, axs = plt.subplots(rows, cols, figsize=(cols*2, rows*2))
    for i in range(rows):
        for j in range(cols):
            idx = i*cols + j
            if idx >= len(images):
                break
            img = images[idx].permute(1, 2, 0).cpu().numpy()
            label = classnames[labels[idx]]
            axs[i, j].imshow(img)
            axs[i, j].set_title(label)
            axs[i, j].axis('off')
    plt.tight_layout()
    plt.show()

show_batch(val_loader, rows=2, cols=8)

In [ ]:
prompts = [f"a photo of a {c}" for c in classnames]
with torch.no_grad():
    text_tokens   = tokenizer(prompts).to(device)
    text_features = model.encode_text(text_tokens)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

correct = total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        image_features = model.encode_image(images)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        # CLIP-style scaled cosine sims
        logits = 100.0 * image_features @ text_features.T
        preds = logits.argmax(dim=-1).cpu()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Zero-shot CIFAR-10 accuracy: {100*correct/total:.2f}%")

In [ ]:
class CoOpPrompt(nn.Module):
    def __init__(self, clip_model, tokenizer, classnames, n_ctx=16, init_scale=0.02, device="cpu"):
        super().__init__()
        self.model = clip_model
        self.tokenizer = tokenizer
        self.classnames = classnames
        self.device = device

        # Text transformer sizes
        self.context_length = getattr(self.model, "context_length", 77)
        self.width = self.model.token_embedding.weight.shape[1]

        # Learnable context (n_ctx continuous "tokens")
        self.ctx = nn.Parameter(init_scale * torch.randn(n_ctx, self.width))

        # Tokenize bare classnames (we'll prepend soft context ourselves)
        with torch.no_grad():
            self.class_token_ids = tokenizer(classnames).to(device)  # [C, L]
            # EOT trick: in CLIP tokenization the EOT id is the largest id in each row
            self.eot_indices = self.class_token_ids.argmax(dim=-1)   # [C]

        # Freeze CLIP
        for p in self.model.parameters():
            p.requires_grad = False

    def forward_text_features(self):
        C = len(self.classnames)
        token_ids = self.class_token_ids  # [C, L]

        # Frozen lookup for class tokens
        with torch.no_grad():
            tok_emb = self.model.token_embedding(token_ids)  # [C, L, W]

        sos = tok_emb[:, :1, :]         # [C,1,W]
        class_part = tok_emb[:, 1:, :]  # [C,L-1,W]

        # Learnable context repeated across classes
        ctx = self.ctx.unsqueeze(0).expand(C, -1, -1)  # [C, n_ctx, W]

        # [SOS] + [CTX...CTX] + [class tokens...]
        x = torch.cat([sos, ctx, class_part], dim=1)  # [C, 1+n_ctx+(L-1), W]

        # Pad/truncate to model context length
        L_target = getattr(self.model, "context_length", 77)
        if x.size(1) > L_target:
            x = x[:, :L_target, :]
        elif x.size(1) < L_target:
            pad_len = L_target - x.size(1)
            pad = torch.zeros(C, pad_len, x.size(2), device=self.device, dtype=x.dtype)
            x = torch.cat([x, pad], dim=1)
        L = x.size(1)

        # Dtypes, pos emb, and (optional) attn mask, all sliced to L
        text_dtype = self.model.token_embedding.weight.dtype
        pos = self.model.positional_embedding[:L].to(text_dtype)  # [L,W]

        attn = getattr(self.model, "attn_mask", None)
        if attn is not None:
            attn = attn[:L, :L].to(text_dtype)  # ensure (L,L)

        # Respect transformer.batch_first
        batch_first = getattr(self.model.transformer, "batch_first", False)
        if batch_first:
            # want [N, L, D]
            x = x.to(text_dtype) + pos.unsqueeze(0)   # [C,L,W]
            x = self.model.transformer(x, attn_mask=attn)  # stays [C,L,W]
            x = self.model.ln_final(x).to(text_dtype)      # [C,L,W]
        else:
            # want [L, N, D]
            x = (x.to(text_dtype) + pos).permute(1, 0, 2)  # [L,C,W]
            x = self.model.transformer(x, attn_mask=attn)  # [L,C,W]
            x = x.permute(1, 0, 2)                         # [C,L,W]
            x = self.model.ln_final(x).to(text_dtype)      # [C,L,W]

        # Pool at EOT (shift by n_ctx), clamp in case of truncation
        eot = (self.eot_indices + self.ctx.shape[0]).clamp(max=L - 1)
        text_emb = x[torch.arange(C, device=self.device), eot] @ self.model.text_projection  # [C,D]

        # Normalize
        text_emb = text_emb / text_emb.norm(dim=-1, keepdim=True)
        return text_emb

In [ ]:
@torch.no_grad()
def evaluate_coop(model, coop, loader, device, loss_fn, desc="eval", pbar=True):
    """
    Eval loop shaped like evaluate.py, adapted to CLIP+CoOp.

    model: frozen CLIP (open_clip)
    coop:  prompt learner (provides forward_text_features())
    loader: DataLoader yielding (images, labels)
    device: torch.device or str
    loss_fn: criterion (CrossEntropyLoss)
    """
    model.eval()
    loss_sum, correct, count = 0.0, 0, 0
    iterator = tqdm(loader, desc=desc, leave=False) if pbar else loader

    # ctx is fixed during eval; text feats can be computed once
    txt_feat = coop.forward_text_features()  # [C, D]

    for images, labels in iterator:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        img_feat = model.encode_image(images)                # [B, D]
        img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)
        logits = 100.0 * (img_feat @ txt_feat.T)             # [B, C]
        loss = loss_fn(logits, labels)

        bs = labels.size(0)
        loss_sum += loss.item() * bs
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        count += bs

        if pbar:
            acc = (correct / max(1, count)) * 100.0
            iterator.set_postfix(loss=loss.item(), acc=f"{acc:.2f}%")

    avg_loss = loss_sum / max(1, count)
    avg_acc = correct / max(1, count)
    return avg_loss, avg_acc


def train_coop(
    model, tokenizer, classnames, train_loader, val_loader, device="cpu",
    n_ctx=16, epochs=5, batch_size=256, lr=5e-3, num_workers=2,
    val_fraction=0.1, log_interval=25, ema_alpha=0.1
):
    """
    Train CoOp prompts on dataset using a tqdm-style loop like training_loop.py.
    Only coop.ctx is optimized; CLIP remains frozen in eval().
    """
    # --- 1) Module & optimizer
    coop = CoOpPrompt(model, tokenizer, classnames, n_ctx=n_ctx, device=device).to(device)
    optimizer = torch.optim.AdamW([coop.ctx], lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # --- 3) Freeze CLIP; we only learn coop.ctx
    model.eval()

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(1, epochs + 1):
        # ---- train epoch (same rhythm as training_loop.py)
        train_sum, train_correct, train_count = 0.0, 0, 0
        ema = None

        iterator = tqdm(
            train_loader,
            desc=f"Epoch {epoch}/{epochs} | train",
            leave=False,
            dynamic_ncols=True,
            mininterval=0.5,
        )

        for step, (images, labels) in enumerate(iterator, start=1):
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            # frozen image features
            with torch.no_grad():
                img_feat = model.encode_image(images)                  # [B, D]
                img_feat = img_feat / img_feat.norm(dim=-1, keepdim=True)

            # prompt-conditioned text features (recomputed as ctx updates)
            txt_feat = coop.forward_text_features()                     # [C, D]
            logits = 100.0 * (img_feat @ txt_feat.T)                    # [B, C]
            loss = loss_fn(logits, labels)

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()

            # stats
            bs = labels.size(0)
            train_sum += loss.item() * bs
            preds = logits.argmax(dim=1)
            train_correct += (preds == labels).sum().item()
            train_count += bs

            # EMA of loss for tqdm postfix
            ema = loss.item() if ema is None else (1 - ema_alpha) * ema + ema_alpha * loss.item()

            if (step % log_interval == 0) or (step == len(iterator)):
                acc_pct = 100.0 * train_correct / max(1, train_count)
                iterator.set_postfix_str(f"loss(ema)={ema:.4f}  acc={acc_pct:.2f}%")

        avg_train_loss = train_sum / max(1, train_count)
        avg_train_acc  = train_correct / max(1, train_count)

        # ---- validation using the separate evaluate (evaluate.py style)
        avg_val_loss, avg_val_acc = evaluate_coop(
            model, coop, val_loader, device, loss_fn=loss_fn, desc="valid", pbar=True
        )

        # ---- log & history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_acc'].append(avg_train_acc)
        history['val_acc'].append(avg_val_acc)

        tqdm.write(
            f"Epoch {epoch:03d}: "
            f"train_loss={avg_train_loss:.4f}  train_acc={avg_train_acc*100:.2f}%  "
            f"val_loss={avg_val_loss:.4f}  val_acc={avg_val_acc*100:.2f}%"
        )

    return coop, history

In [ ]:
coop, history = train_coop(
    model=model,
    tokenizer=tokenizer,
    classnames=classnames,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    n_ctx=cfg.prompt_len,
    epochs=cfg.max_epochs,
    batch_size=cfg.batch_size,
    lr=cfg.lr_prompt,
    num_workers=cfg.num_workers,
)